In [1]:
from google.cloud import bigquery
import pandas as pd 
import chime   
import seaborn as sns
client = bigquery.Client("gcp-wow-rwds-ai-checkout-dev")

/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [16]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

In [143]:
check = client.query(
"""
SELECT logs.*, ARRAY_AGG(DISTINCT cc.media_type IGNORE NULLS ORDER BY cc.media_type) AS media_types
FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3` logs
LEFT JOIN `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` cc 
    ON logs.campaign_id = cc.booking_number
--WHERE query_start_time >= "2024-08-13 21:45:24.626167" 
-- campaign_id <> "WOW20000952"
WHERE query_step = "10"
GROUP BY ALL
ORDER BY query_end_time
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,campaign_id,query_step,query_type,query_start_time,query_end_time,query_duration_in_seconds,query_duration_in_minutes,media_types
0,WOW20003763,10,Total Runtime,2024-08-14 11:48:46.217942,2024-08-14 11:49:00.635446,654,10.9,[Display Aisle and Category]


In [141]:
check = client.query(
"""
SELECT logs.*
FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3` logs
ORDER BY query_end_time
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,campaign_id,query_step,query_type,query_start_time,query_end_time,query_duration_in_seconds,query_duration_in_minutes
0,WOW20006196,1,Adobe Web Events,2024-08-14 11:32:03.310670,2024-08-14 11:32:25.801494,22,0.366667
1,WOW20006196,2,Campaign Dates,2024-08-14 11:32:30.163342,2024-08-14 11:32:39.410414,9,0.150000
2,WOW20006196,3,Campaign Pages,2024-08-14 11:32:42.770175,2024-08-14 11:32:55.727210,13,0.216667
3,WOW20006196,4,Exposed Shoppers,2024-08-14 11:33:00.181809,2024-08-14 11:33:14.907617,14,0.233333
4,WOW20003763,1,Adobe Web Events,2024-08-14 11:38:06.407214,2024-08-14 11:38:20.514603,14,0.233333
5,WOW20003763,2,Campaign Dates,2024-08-14 11:38:24.757396,2024-08-14 11:38:43.923513,19,0.316667
6,WOW20003763,3,Campaign Pages,2024-08-14 11:38:50.490890,2024-08-14 11:39:14.925255,24,0.400000
7,WOW20003763,4,Exposed Shoppers,2024-08-14 11:39:23.167356,2024-08-14 11:39:43.434593,20,0.333333
8,WOW20003763,5,Non-Exposed Shoppers,2024-08-14 11:39:46.799807,2024-08-14 11:40:05.662786,19,0.316667
9,WOW20003763,6,Exposed Transactions,2024-08-14 11:40:09.189429,2024-08-14 11:42:12.991168,123,2.050000


In [ ]:
check = client.query(
"""
with step_one AS (
    SELECT DISTINCT 
        booking_number,
        media_type,
        CASE WHEN media_type <> "Display Aisle and Category" THEN 1 ELSE 0 END AS non_display_aisle_asset
    FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` 
    ORDER BY booking_number
)
SELECT DISTINCT cc.* EXCEPT(individual_product_string)
FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` cc 
LEFT JOIN (SELECT DISTINCT booking_number FROM step_one WHERE non_display_aisle_asset = 1) non
    ON non.booking_number = cc.booking_number 
WHERE non.booking_number IS NULL
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

In [134]:
check = client.query(
"""
SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_one_events` WHERE booking_id = "WOW20002352"  LIMIT 20
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,date_time,shopper_id,crn,page_name,site_platform,catalogue_flag,homepage_flag,booking_id,ci_id,post_evar130,search_term1,search_term2,search_term3,search_term_edit,media_type,campaign_start_date,campaign_end_date,media_start_date,media_end_date,quoteline_sku,original_media_type
0,2023-10-14 00:48:00+00:00,None,3300000000007711583,ww-sm:shop:browse:drinks:flavoured-milk,Web,0,0,WOW20002352,CI-10076737,impression,None,None,None,None,Display Aisle and Category,2023-10-04,2023-10-31,2023-10-04,2023-10-17,309672,Display Aisle and Category
1,2023-10-16 13:51:19+00:00,None,1100000000097582677,ww-sm:shop:browse:drinks:water:still-water,Web,0,0,WOW20002352,CI-10076737,impression,None,None,None,None,Display Aisle and Category,2023-10-04,2023-10-31,2023-10-04,2023-10-17,309672,Display Aisle and Category
2,2023-10-08 14:24:36+00:00,None,3300000000004761622,ww-sm:shop:browse:drinks:coffee:coffee-specials,Web,0,0,WOW20002352,CI-10076737,impression,None,None,None,None,Display Aisle and Category,2023-10-04,2023-10-31,2023-10-04,2023-10-17,309672,Display Aisle and Category
3,2023-10-06 15:38:56+00:00,None,1100000000130041710,ww-sm:shop:browse:drinks:long-life-milk:powdered-milk,Web,0,0,WOW20002352,CI-10076737,impression,None,None,None,None,Display Aisle and Category,2023-10-04,2023-10-31,2023-10-04,2023-10-17,309672,Display Aisle and Category
4,2023-10-09 16:23:23+00:00,None,1100000000003656043,ww-sm:shop:browse:drinks:water:sparkling-water,Web,0,0,WOW20002352,CI-10076737,impression,None,None,None,None,Display Aisle and Category,2023-10-04,2023-10-31,2023-10-04,2023-10-17,309672,Display Aisle and Category
5,2023-10-08 09:21:45+00:00,None,3300000000004237448,ww-sm:shop:browse:drinks:soft-drinks:soft-drink-cans,Web,0,0,WOW20002352,CI-10076737,impression,None,None,None,None,Display Aisle and Category,2023-10-04,2023-10-31,2023-10-04,2023-10-17,309672,Display Aisle and Category
6,2023-10-11 16:13:06+00:00,None,3300000000007813244,ww-sm:shop:browse:drinks:soft-drinks:soft-drink-bottles,Web,0,0,WOW20002352,CI-10076737,impression,None,None,None,None,Display Aisle and Category,2023-10-04,2023-10-31,2023-10-04,2023-10-17,309672,Display Aisle and Category
7,2023-10-05 20:33:48+00:00,None,1100000000083221639,ww-sm:shop:browse:drinks:coffee:coffee-capsules,Web,0,0,WOW20002352,CI-10076737,impression,None,None,None,None,Display Aisle and Category,2023-10-04,2023-10-31,2023-10-04,2023-10-17,309672,Display Aisle and Category
8,2023-10-14 12:43:50+00:00,None,1000000000001208250,ww-sm:shop:browse:drinks:coffee:coffee-capsules,Web,0,0,WOW20002352,CI-10076737,impression,None,None,None,None,Display Aisle and Category,2023-10-04,2023-10-31,2023-10-04,2023-10-17,309672,Display Aisle and Category
9,2023-10-17 09:06:56+00:00,None,1000000000001760963,ww-sm:shop:browse:drinks:cordials-juices-iced-teas,Web,0,0,WOW20002352,CI-10076737,impression,None,None,None,None,Display Aisle and Category,2023-10-04,2023-10-31,2023-10-04,2023-10-17,309672,Display Aisle and Category


In [131]:
check = client.query(
"""
SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_three_page_names` WHERE booking_id = "WOW20002352"  LIMIT 5
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,booking_id,campaign_start_date,campaign_end_date,media_start_date,media_end_date,site_platform,media_type,ci_id,search_term3,page_name,post_evar130,skus,descriptions
0,WOW20002352,2023-10-04,2023-10-31,2023-10-04,2023-10-17,Web,Display Aisle and Category,CI-10076737,None,ww-sm:shop:productdetails:310675:vitasoy-oat-milk,impression,[309672],[Lavazza Barista Organic Beans 1kg]
1,WOW20002352,2023-10-04,2023-10-31,2023-10-04,2023-10-17,Web,Display Aisle and Category,CI-10076737,None,ww-sm:shop:browse:drinks:cordials-juices-iced-teas:cordials-juices-iced-teas-specials,impression,[309672],[Lavazza Barista Organic Beans 1kg]
2,WOW20002352,2023-10-04,2023-10-31,2023-10-04,2023-10-17,Web,Display Aisle and Category,CI-10076737,None,ww-sm:shop:browse:drinks:flavoured-milk:bubble-tea,impression,[309672],[Lavazza Barista Organic Beans 1kg]
3,WOW20002352,2023-10-04,2023-10-31,2023-10-04,2023-10-17,Web,Display Aisle and Category,CI-10076737,None,ww-sm:shop:browse:drinks:sports-energy-drinks:sports-drinks,impression,[309672],[Lavazza Barista Organic Beans 1kg]
4,WOW20002352,2023-10-04,2023-10-31,2023-10-04,2023-10-17,Web,Display Aisle and Category,CI-10076737,None,ww-sm:shop:browse:drinks:tea:herbal-specialty-tea,impression,[309672],[Lavazza Barista Organic Beans 1kg]


In [172]:
check = client.query(
"""
with step_one AS (
    SELECT final.*, skus
    FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results` final
    LEFT JOIN (SELECT DISTINCT campaign_id FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`) logs 
        ON final.booking_id = logs.campaign_id 
    LEFT JOIN (SELECT booking_number, campaign_start_date, campaign_end_date, ARRAY_AGG(DISTINCT individual_product_string ORDER BY individual_product_string) AS skus FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` GROUP BY 1,2,3) cc
        ON cc.booking_number = final.booking_id 
    WHERE logs.campaign_id IS NOT NULL OR booking_id = "WOW20002352" 
    ORDER BY booking_id, exposure_context
),
summary AS (
    SELECT 
        booking_id,
        ARRAY_TO_STRING(media_types, ",") AS media_types,
        exposure_context, 
        COUNT(DISTINCT exposure_crn) AS visitors,
        COUNT(DISTINCT converted_crn) AS buyers,
        SAFE_DIVIDE(COUNT(DISTINCT converted_crn) , COUNT(DISTINCT exposure_crn)) AS conversion_rate,
        SAFE_DIVIDE(COUNT(DISTINCT basket_key) , COUNT(DISTINCT converted_crn)) AS baskets_per_purchased_shopper, 
        SAFE_DIVIDE(COUNT(DISTINCT basket_key) , COUNT(DISTINCT exposure_crn)) AS baskets_per_visited_shopper,
        SAFE_DIVIDE(SUM(tot_net_incld_gst) , COUNT(DISTINCT exposure_crn)) AS spend_per_visited_shopper,
        SAFE_DIVIDE(SUM(tot_net_incld_gst) , COUNT(DISTINCT converted_crn)) AS spend_per_purchased_shopper 
    FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_eight_distinct_transactions`  
    WHERE booking_id IN (SELECT DISTINCT booking_id FROM step_one)
    GROUP BY 1,2,3
),
exposed AS (
    SELECT * FROM summary WHERE exposure_context = "Exposed" 
),
not_exposed AS (
    SELECT * FROM summary WHERE exposure_context = "Not Exposed" 
),
wide_format AS (
SELECT 
    exposed.booking_id, 
    exposed.media_types,
    exposed.visitors AS exposed_visitors,
    not_exposed.visitors AS not_exposed_visitors,
    exposed.buyers AS exposed_buyers,
    not_exposed.buyers AS not_exposed_buyers,
    ROUND((exposed.conversion_rate * 100), 2) AS exposed_conversion_rate,
    ROUND((not_exposed.conversion_rate * 100), 2) AS not_exposed_conversion_rate,
    ROUND(exposed.baskets_per_purchased_shopper, 2) AS baskets_per_exposed_buyer,
    ROUND(not_exposed.baskets_per_purchased_shopper, 2) AS baskets_per_not_exposed_buyer,
    ROUND(exposed.baskets_per_visited_shopper, 2) AS baskets_per_exposed_visitor,
    ROUND(not_exposed.baskets_per_visited_shopper, 2) AS baskets_per_not_exposed_visitor,
    ROUND(exposed.spend_per_visited_shopper, 2) AS spend_per_exposed_visitor,
    ROUND(not_exposed.spend_per_visited_shopper, 2) AS spend_per_not_exposed_visitor,
    ROUND(exposed.spend_per_purchased_shopper, 2) AS spend_per_exposed_buyer,
    ROUND(not_exposed.spend_per_purchased_shopper, 2) AS spend_per_not_exposed_buyer
FROM exposed 
LEFT JOIN not_exposed 
    ON exposed.booking_id = not_exposed.booking_id
    AND exposed.media_types = not_exposed.media_types
),
percentages AS (
SELECT 
    *,
    SAFE_DIVIDE(exposed_conversion_rate , not_exposed_conversion_rate) - 1 AS conversion_rate_perc_diff,
    exposed_conversion_rate - not_exposed_conversion_rate AS conversion_rate_pp_diff, 
    SAFE_DIVIDE(baskets_per_exposed_buyer , baskets_per_not_exposed_buyer) - 1 AS baskets_per_buyer_perc_diff,
    baskets_per_exposed_buyer - baskets_per_not_exposed_buyer AS baskets_per_buyer_pp_diff, 
    SAFE_DIVIDE(baskets_per_exposed_visitor , baskets_per_not_exposed_visitor) - 1 AS baskets_per_visitor_perc_diff, 
    baskets_per_exposed_visitor - baskets_per_not_exposed_visitor AS baskets_per_visitor_pp_diff, 
    SAFE_DIVIDE(spend_per_exposed_visitor , spend_per_not_exposed_visitor) - 1 AS spend_per_visitor_perc_diff,
    spend_per_exposed_visitor - spend_per_not_exposed_visitor AS spend_per_visitor_pp_diff,
    SAFE_DIVIDE(spend_per_exposed_buyer , spend_per_not_exposed_buyer) - 1 AS spend_per_buyer_perc_diff,
    spend_per_exposed_buyer - spend_per_not_exposed_buyer AS spend_per_buyer_pp_diff
FROM wide_format
WHERE exposed_visitors >= 2000
AND not_exposed_visitors >= 2000
ORDER BY booking_id, media_types
) 
    SELECT 
        media_types, 
        COUNT(DISTINCT booking_id) AS n_campaigns,
        AVG(conversion_rate_perc_diff) AS avg_conversion_perc_diff,
        --AVG(conversion_rate_pp_diff) AS avg_conversion_pp_diff, 
        AVG(baskets_per_buyer_perc_diff) AS avg_baskets_per_buyer_perc_diff,
        --AVG(baskets_per_buyer_pp_diff) AS avg_baskets_per_buyer_pp_diff,
        AVG(baskets_per_visitor_perc_diff) AS avg_baskets_per_visitor_perc_diff,
        --AVG(baskets_per_visitor_pp_diff) AS avg_baskets_per_visitor_pp_diff,
        AVG(spend_per_visitor_perc_diff) AS avg_spend_per_visitor_perc_diff,
        --AVG(spend_per_visitor_pp_diff) AS avg_spend_per_visitor_pp_diff,
        AVG(spend_per_buyer_perc_diff) AS avg_spend_per_buyer_perc_diff
        --AVG(spend_per_buyer_pp_diff) AS avg_spend_per_buyer_pp_diff
    FROM percentages
    GROUP BY 1
    ORDER BY 2 DESC, 3 DESC
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,media_types,n_campaigns,avg_conversion_perc_diff,avg_baskets_per_buyer_perc_diff,avg_baskets_per_visitor_perc_diff,avg_spend_per_visitor_perc_diff,avg_spend_per_buyer_perc_diff
0,Display Aisle and Category,8,0.320476,0.013084,-0.652174,0.380913221,0.002869244
1,Display Search,1,0.854167,0.038095,0.000000,1.500000000,0.292584009
2,Display Reach,1,-0.692737,0.028571,-0.500000,-0.592592593,0.291859696
3,"Display Reach,Display Search",1,-0.750000,-0.099099,-1.000000,-0.833333333,-0.206047032


In [181]:
check = client.query(
"""
with step_one AS (
    SELECT final.*, skus
    FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results` final
    LEFT JOIN (SELECT DISTINCT campaign_id FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3`) logs 
        ON final.booking_id = logs.campaign_id 
    LEFT JOIN (SELECT booking_number, campaign_start_date, campaign_end_date, ARRAY_AGG(DISTINCT individual_product_string ORDER BY individual_product_string) AS skus FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` GROUP BY 1,2,3) cc
        ON cc.booking_number = final.booking_id 
    WHERE logs.campaign_id IS NOT NULL OR booking_id = "WOW20002352" 
    ORDER BY booking_id, exposure_context
),
summary AS (
    SELECT 
        booking_id,
        ARRAY_TO_STRING(media_types, ",") AS media_types,
        exposure_context, 
        COUNT(DISTINCT exposure_crn) AS visitors,
        COUNT(DISTINCT converted_crn) AS buyers,
        SAFE_DIVIDE(COUNT(DISTINCT converted_crn) , COUNT(DISTINCT exposure_crn)) AS conversion_rate,
        SAFE_DIVIDE(COUNT(DISTINCT basket_key) , COUNT(DISTINCT converted_crn)) AS baskets_per_purchased_shopper, 
        SAFE_DIVIDE(COUNT(DISTINCT basket_key) , COUNT(DISTINCT exposure_crn)) AS baskets_per_visited_shopper,
        SAFE_DIVIDE(SUM(tot_net_incld_gst) , COUNT(DISTINCT exposure_crn)) AS spend_per_visited_shopper,
        SAFE_DIVIDE(SUM(tot_net_incld_gst) , COUNT(DISTINCT converted_crn)) AS spend_per_purchased_shopper 
    FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_eight_distinct_transactions`  
    WHERE booking_id IN (SELECT DISTINCT booking_id FROM step_one)
    GROUP BY 1,2,3
),
exposed AS (
    SELECT * FROM summary WHERE exposure_context = "Exposed" 
),
not_exposed AS (
    SELECT * FROM summary WHERE exposure_context = "Not Exposed" 
),
wide_format AS (
SELECT 
    exposed.booking_id, 
    exposed.media_types,
    exposed.visitors AS exposed_visitors,
    not_exposed.visitors AS not_exposed_visitors,
    exposed.buyers AS exposed_buyers,
    not_exposed.buyers AS not_exposed_buyers,
    ROUND(exposed.conversion_rate, 2) AS exposed_conversion_rate,
    ROUND(not_exposed.conversion_rate, 2) AS not_exposed_conversion_rate,
    ROUND(exposed.baskets_per_purchased_shopper, 2) AS baskets_per_exposed_buyer,
    ROUND(not_exposed.baskets_per_purchased_shopper, 2) AS baskets_per_not_exposed_buyer,
    ROUND(exposed.baskets_per_visited_shopper, 2) AS baskets_per_exposed_visitor,
    ROUND(not_exposed.baskets_per_visited_shopper, 2) AS baskets_per_not_exposed_visitor,
    ROUND(exposed.spend_per_visited_shopper, 2) AS spend_per_exposed_visitor,
    ROUND(not_exposed.spend_per_visited_shopper, 2) AS spend_per_not_exposed_visitor,
    ROUND(exposed.spend_per_purchased_shopper, 2) AS spend_per_exposed_buyer,
    ROUND(not_exposed.spend_per_purchased_shopper, 2) AS spend_per_not_exposed_buyer
FROM exposed 
LEFT JOIN not_exposed 
    ON exposed.booking_id = not_exposed.booking_id
    AND exposed.media_types = not_exposed.media_types
)
SELECT 
    *,
    SAFE_DIVIDE(exposed_conversion_rate , not_exposed_conversion_rate) - 1 AS conversion_rate_perc_diff,
    exposed_conversion_rate - not_exposed_conversion_rate AS conversion_rate_pp_diff, 
    SAFE_DIVIDE(baskets_per_exposed_buyer , baskets_per_not_exposed_buyer) - 1 AS baskets_per_buyer_perc_diff,
    baskets_per_exposed_buyer - baskets_per_not_exposed_buyer AS baskets_per_buyer_pp_diff, 
    SAFE_DIVIDE(baskets_per_exposed_visitor , baskets_per_not_exposed_visitor) - 1 AS baskets_per_visitor_perc_diff, 
    baskets_per_exposed_visitor - baskets_per_not_exposed_visitor AS baskets_per_visitor_pp_diff, 
    SAFE_DIVIDE(spend_per_exposed_visitor , spend_per_not_exposed_visitor) - 1 AS spend_per_visitor_perc_diff,
    spend_per_exposed_visitor - spend_per_not_exposed_visitor AS spend_per_visitor_pp_diff,
    SAFE_DIVIDE(spend_per_exposed_buyer , spend_per_not_exposed_buyer) - 1 AS spend_per_buyer_perc_diff,
    spend_per_exposed_buyer - spend_per_not_exposed_buyer AS spend_per_buyer_pp_diff
FROM wide_format
WHERE exposed_visitors >= 2000
AND not_exposed_visitors >= 2000 
#AND media_types = "Display Aisle and Category"
ORDER BY booking_id, media_types
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df.to_csv("~/Downloads/DX New Exposure Based Test vs Control - Display Aisle and Category.csv")
check_df

,booking_id,media_types,exposed_visitors,not_exposed_visitors,exposed_buyers,not_exposed_buyers,exposed_conversion_rate,not_exposed_conversion_rate,baskets_per_exposed_buyer,baskets_per_not_exposed_buyer,baskets_per_exposed_visitor,baskets_per_not_exposed_visitor,spend_per_exposed_visitor,spend_per_not_exposed_visitor,spend_per_exposed_buyer,spend_per_not_exposed_buyer,conversion_rate_perc_diff,conversion_rate_pp_diff,baskets_per_buyer_perc_diff,baskets_per_buyer_pp_diff,baskets_per_visitor_perc_diff,baskets_per_visitor_pp_diff,spend_per_visitor_perc_diff,spend_per_visitor_pp_diff,spend_per_buyer_perc_diff,spend_per_buyer_pp_diff
0,WOW20002352,Display Aisle and Category,10535,298101,0,35,0.00,0.00,NaN,1.03,0.00,0.00,None,0E-9,None,29.550000000,NaN,0.00,NaN,NaN,NaN,0.00,None,None,None,None
1,WOW20003763,Display Aisle and Category,15441,481437,3913,124941,0.25,0.26,1.27,1.77,0.32,0.46,1.330000000,1.920000000,5.250000000,7.410000000,-0.038462,-0.01,-0.282486,-0.50,-0.304348,-0.14,-0.307291667,-0.590000000,-0.291497976,-2.160000000
2,WOW20004117,Display Aisle and Category,4542,135868,23,273,0.01,0.00,1.00,1.06,0.01,0.00,0.030000000,0.020000000,6.230000000,9.300000000,NaN,0.01,-0.056604,-0.06,NaN,0.01,0.500000000,0.010000000,-0.330107527,-3.070000000
3,WOW20004299,Display Aisle and Category,2986,152411,5,98,0.00,0.00,1.20,1.04,0.00,0.00,0.020000000,0.010000000,12.480000000,14.190000000,NaN,0.00,0.153846,0.16,NaN,0.00,1.000000000,0.010000000,-0.120507400,-1.710000000
4,WOW20004519,Display Aisle and Category,13297,432952,37,1173,0.00,0.00,1.03,1.13,0.00,0.00,0.030000000,0.040000000,12.180000000,13.090000000,NaN,0.00,-0.088496,-0.10,NaN,0.00,-0.250000000,-0.010000000,-0.069518717,-0.910000000
5,WOW20004853,Display Aisle and Category,6262,667019,11,1238,0.00,0.00,1.36,1.09,0.00,0.00,0.050000000,0.040000000,28.550000000,22.970000000,NaN,0.00,0.247706,0.27,NaN,0.00,0.250000000,0.010000000,0.242925555,5.580000000
6,WOW20004864,Display Aisle and Category,3506,282734,10,454,0.00,0.00,1.10,1.05,0.00,0.00,0.030000000,0.010000000,12.100000000,8.680000000,NaN,0.00,0.047619,0.05,NaN,0.00,2.000000000,0.020000000,0.394009217,3.420000000
7,WOW20005100,Display Aisle and Category,31129,643532,614,16772,0.02,0.03,1.10,1.13,0.02,0.03,0.440000000,0.630000000,22.540000000,24.240000000,-0.333333,-0.01,-0.026549,-0.03,-0.333333,-0.01,-0.301587302,-0.190000000,-0.070132013,-1.700000000
8,WOW20005261,Display Aisle and Category,20549,2926,94,33,0.00,0.01,1.07,1.00,0.00,0.01,0.090000000,0.190000000,19.690000000,16.480000000,-1.000000,-0.01,0.070000,0.07,-1.000000,-0.01,-0.526315789,-0.100000000,0.194781553,3.210000000
9,WOW20005261,Display Reach,49038,28279,270,505,0.01,0.02,1.08,1.05,0.01,0.02,0.110000000,0.270000000,19.520000000,15.110000000,-0.500000,-0.01,0.028571,0.03,-0.500000,-0.01,-0.592592593,-0.160000000,0.291859696,4.410000000


In [136]:
check = client.query(
"""
SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_five_non_exposed_shoppers` WHERE booking_id = "WOW20002352" AND crn IS NOT NULL LIMIT 5
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,booking_id,crn,non_exposure_time,media_type
0,WOW20002352,1000000000000000143,2023-10-12 18:01:31+00:00,Display Aisle and Category
1,WOW20002352,1000000000000000206,2023-10-07 15:51:52+00:00,Display Aisle and Category
2,WOW20002352,1000000000000000206,2023-10-07 15:50:07+00:00,Display Aisle and Category
3,WOW20002352,1000000000000000432,2023-10-07 18:32:44+00:00,Display Aisle and Category
4,WOW20002352,1000000000000000432,2023-10-07 18:48:22+00:00,Display Aisle and Category


In [116]:
check = client.query(
"""
SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_seven_non_exposed_campaign_purchases`  WHERE booking_id = "WOW20002352" LIMIT 5
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,impression_date,booking_id,media_type,exposure_context,exposure_crn,order_context,converted_shopper_identification_number,start_txn_time,basket_key,article,tot_net_incld_gst,exposure_to_purchase_window


In [137]:
check = client.query(
"""
with 

current_campaigns AS (
    SELECT DISTINCT campaign_id 
    FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3` 
    --WHERE query_start_time >= "2024-08-13 21:45:24.626167" 
    WHERE query_step = "10"
),
exposed AS (
    SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results` WHERE exposure_context = "Exposed" 
),
not_exposed AS (
    SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results` WHERE exposure_context = "Not Exposed" 
)
SELECT final.*, cc.campaign_start_date, cc.campaign_end_date, cc.skus
FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_nine_final_results` final
INNER JOIN exposed 
    ON final.booking_id = exposed.booking_id 
    AND final.media_types = exposed.media_types
INNER JOIN not_exposed 
    ON final.booking_id = not_exposed.booking_id 
    AND final.media_types = not_exposed.media_types
LEFT JOIN (SELECT booking_number, campaign_start_date, campaign_end_date, ARRAY_AGG(DISTINCT individual_product_string ORDER BY individual_product_string) AS skus FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` GROUP BY 1,2,3) cc
    ON cc.booking_number = final.booking_id 
INNER JOIN current_campaigns 
    ON final.booking_id = current_campaigns.campaign_id
--WHERE LOWER(final.media_types) NOT LIKE "%search%"
ORDER BY final.booking_id, final.media_types, final.exposure_context
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,booking_id,media_types,exposure_context,visitors,buyers,conversion_rate,baskets_per_purchased_shopper,baskets_per_visited_shopper,spend_per_visited_shopper,spend_per_purchased_shopper,campaign_start_date,campaign_end_date,skus
0,WOW20002352,Display Aisle and Category,Exposed,10535,0,0.000000,NaN,NaN,None,None,2023-10-04,2023-10-31,[309672]
1,WOW20002352,Display Aisle and Category,Not Exposed,298101,35,0.000117,1.028571,1.028571,0.003469864,29.553428571,2023-10-04,2023-10-31,[309672]


In [140]:
check = client.query(
"""
        with step_one AS (
            SELECT DISTINCT 
                booking_number,
                media_type,
                CASE WHEN media_type <> "Display Aisle and Category" THEN 1 ELSE 0 END AS non_display_aisle_asset
            FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` 
            ORDER BY booking_number
        ),
        step_two AS (
        SELECT cc.* EXCEPT(individual_product_string), ARRAY_AGG(DISTINCT individual_product_string IGNORE NULLS ORDER BY individual_product_string) AS skus
        FROM `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` cc 
        LEFT JOIN (SELECT DISTINCT booking_number FROM step_one WHERE non_display_aisle_asset = 1) non
            ON non.booking_number = cc.booking_number 
        WHERE non.booking_number IS NULL
        GROUP BY ALL
        )
        SELECT ARRAY_AGG(DISTINCT booking_number) FROM step_two WHERE ARRAY_LENGTH(skus) >= 5
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,booking_number,campaign_start_date,campaign_end_date,line_name,media_type,media_start_date,media_end_date,total_media_spend,skus
0,WOW20002352,2023-10-04,2023-10-31,CI-10076736,Display Aisle and Category,2023-10-04,2023-10-17,6000.00,[309672]
1,WOW20002352,2023-10-04,2023-10-31,CI-10076737,Display Aisle and Category,2023-10-04,2023-10-17,6000.00,[309672]
2,WOW20002352,2023-10-04,2023-10-31,CI-10076738,Display Aisle and Category,2023-10-04,2023-10-17,6000.00,[309672]
3,WOW20002875,2023-12-06,2023-12-19,CI-10077352,Display Aisle and Category,2023-12-06,2023-12-12,3000.00,"[726937, 727013]"
4,WOW20002875,2023-12-06,2023-12-19,CI-10077353,Display Aisle and Category,2023-12-06,2023-12-12,3000.00,"[726937, 727013]"
5,WOW20002875,2023-12-06,2023-12-19,CI-10077354,Display Aisle and Category,2023-12-06,2023-12-12,3000.00,"[726937, 727013]"
6,WOW20003763,2023-07-26,2023-11-21,CI-10064811,Display Aisle and Category,2023-11-15,2023-11-21,2700.00,"[106218, 133211, 133390, 157649, 180148, 508279]"
7,WOW20003763,2023-07-26,2023-11-21,CI-10064812,Display Aisle and Category,2023-11-15,2023-11-21,2700.00,"[106218, 133211, 133390, 157649, 180148, 508279]"
8,WOW20003763,2023-07-26,2023-11-21,CI-10064813,Display Aisle and Category,2023-11-15,2023-11-21,2700.00,"[106218, 133211, 133390, 157649, 180148, 508279]"
9,WOW20003763,2023-07-26,2023-11-21,CI-10064814,Display Aisle and Category,2023-10-04,2023-10-10,2700.00,"[106218, 133211, 133390, 157649, 180148, 508279]"


In [ ]:
check = client.query(
"""
with 

309672
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,booking_id,media_types,exposure_context,visitors,buyers,conversion_rate,baskets_per_purchased_shopper,baskets_per_visited_shopper,spend_per_visited_shopper,spend_per_purchased_shopper,campaign_start_date,campaign_end_date,skus
0,WOW20002352,Display Aisle and Category,Exposed,10535,0,0.000000,NaN,NaN,None,None,2023-10-04,2023-10-31,[309672]
1,WOW20002352,Display Aisle and Category,Not Exposed,298101,35,0.000117,1.028571,1.028571,0.003469864,29.553428571,2023-10-04,2023-10-31,[309672]


In [53]:
check = client.query(
"""
with current_campaigns AS (
    SELECT DISTINCT campaign_id 
    FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3` 
    WHERE query_start_time >= "2024-08-13 21:45:24.626167" 
    AND query_step = "10"
),
exposed AS (
    SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.new_dx_exposure_method_step_three_page_names` 
)
SELECT exposed.* 
FROM exposed
INNER JOIN current_campaigns 
    ON exposed.booking_id = current_campaigns.campaign_id
--ORDER BY final.booking_id, final.media_types, final.exposure_context
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,booking_id,campaign_start_date,campaign_end_date,media_start_date,media_end_date,site_platform,media_type,ci_id,search_term3,page_name,post_evar130,skus,descriptions


In [50]:
check = client.query(
"""
CREATE OR REPLACE TABLE `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3` AS 
    SELECT * FROM `gcp-wow-cart-data-dev-d4d7.davide.cartology_dx_logs_3` WHERE campaign_id = "giraffe"
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

""


In [68]:
check = client.query(
"""
SELECT 
    cc.media_type,
    COUNT(DISTINCT CONCAT(events.booking_id, events.ci_id)) AS n_assets
FROM `gcp-wow-cart-data-dev-d4d7.davide.adobe_carto_catalogue_homepage_ciid_search_term_test` events
LEFT JOIN `gcp-wow-cart-data-dev-d4d7.davide.carto_campaigns` cc
    ON events.booking_id = cc.booking_number 
    AND events.ci_id = cc.line_name
WHERE media_type IS NOT NULL
GROUP BY 1
"""
).result()
check_df = check.to_dataframe()
chime.success()
check_df 

,media_type,n_assets
0,Display Reach,109
1,Display Recipes,31
2,Display Aisle and Category,115
3,Results Page Display,6
4,Display Search,344
5,Display Customer Segment,126
6,Display Specials,11
